In [1]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [89]:
print("Data read started...")
data = pd.read_csv("result_nback.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(828, 11)


# Eliminate EEG data

In [90]:
def feature_selection(data, feature):
    if(feature == "ECG_Burg"):
        data = np.delete(data, 6, 1) 
        data = np.delete(data, 5, 1) 
        data = np.delete(data, 3, 1) 
        data = np.delete(data, 2, 1) 
    if(feature == "ECG_TimeSeries"):
        data = np.delete(data, 6, 1) 
        data = np.delete(data, 5, 1) 
        data = np.delete(data, 3, 1) 
        data = np.delete(data, 1, 1) 
    if(feature == "ECG_SDNN"):
        data = np.delete(data, 6, 1) 
        data = np.delete(data, 5, 1) 
        data = np.delete(data, 2, 1) 
        data = np.delete(data, 1, 1) 
    if(feature == "ECG_SDSD"):
        data = np.delete(data, 6, 1) 
        data = np.delete(data, 3, 1) 
        data = np.delete(data, 2, 1) 
        data = np.delete(data, 1, 1) 
    if(feature == "ECG_RMSSD"):
        data = np.delete(data, 5, 1) 
        data = np.delete(data, 3, 1) 
        data = np.delete(data, 2, 1) 
        data = np.delete(data, 1, 1) 
    
    return data

In [91]:
for i in range (1,5):
    data = np.delete(data, 1, 1) 

data = feature_selection(data, "ECG_SDNN")
data.shape

(828, 3)

In [92]:
data[1]

array(['nback gyakorlas', 352.6, 4.9275], dtype=object)

# Dictionary for the levels

In [93]:
level=["nback gyakorlas","0back","1back","2back","3back"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [94]:
for i in range (1,3):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [95]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [96]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]


# Shuffle data

In [97]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [98]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [99]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(579, 2) (579, 5) (124, 2) (124, 5) (125, 2) (125, 5)


# Build the net

In [100]:
model = Sequential()

model.add(Dense(200, input_shape=(2,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [101]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 579 samples, validate on 124 samples
Epoch 1/150
579/579 [==============================] - 0s - loss: 1.5679 - acc: 0.3506 - val_loss: 1.4750 - val_acc: 0.4113
Epoch 2/150
579/579 [==============================] - 0s - loss: 1.4795 - acc: 0.4059 - val_loss: 1.4130 - val_acc: 0.4032
Epoch 3/150
579/579 [==============================] - 0s - loss: 1.4345 - acc: 0.4335 - val_loss: 1.3726 - val_acc: 0.3871
Epoch 4/150
579/579 [==============================] - 0s - loss: 1.4080 - acc: 0.4335 - val_loss: 1.3499 - val_acc: 0.4032
Epoch 5/150
579/579 [==============================] - 0s - loss: 1.3990 - acc: 0.4145 - val_loss: 1.3165 - val_acc: 0.4677
Epoch 6/150
579/579 [==============================] - 0s - loss: 1.3696 - acc: 0.4508 - val_loss: 1.2958 - val_acc: 0.4435
Epoch 7/150
579/579 [==============================] - 0s - loss: 1.3640 - acc: 0.4370 - val_loss: 1.2958 - val_acc: 0.4274
Epoch 8/150
579/579 [==============================] - 0s - loss: 1.3426 - acc: 0.4387

In [102]:
model.evaluate(x_test, y_test, batch_size=batch_size)

 50/125 [===========>..................] - ETA: 0s

[1.0507098436355591, 0.54399999380111697]

In [33]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(nback gyakorlas)', 'class 1(0back)', 'class 2(1back)', 'class 3(2back)', 'class 4(3back)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

125/125 [==============================] - 0s     
[2 0 3 3 3 0 1 1 2 4 0 2 1 1 3 1 3 4 3 2 0 0 4 4 3 4 0 1 1 4 1 4 0 1 1 2 2
 2 3 0 1 1 2 4 0 4 2 0 1 1 2 4 0 1 4 3 1 3 1 1 4 3 1 2 4 2 4 0 2 3 4 4 2 4
 0 3 4 3 2 1 1 4 4 4 1 0 1 1 2 2 1 2 3 4 4 2 4 1 1 3 1 4 2 3 2 4 0 4 0 0 1
 4 3 3 3 1 3 1 1 3 1 3 2 4 3]
 32/125 [======>.......................] - ETA: 0s                          precision    recall  f1-score   support

class 0(nback gyakorlas)       0.35      0.30      0.32        20
          class 1(0back)       0.24      0.42      0.31        19
          class 2(1back)       0.32      0.35      0.33        20
          class 3(2back)       0.54      0.48      0.51        27
          class 4(3back)       0.72      0.54      0.62        39

             avg / total       0.49      0.44      0.45       125

[[ 6  7  3  4  0]
 [ 3  8  6  1  1]
 [ 0  4  7  2  7]
 [ 5  5  4 13  0]
 [ 3  9  2  4 21]]
